In [2]:
import faiss
import torch
import wget
import os
import numpy as np
import pandas as pd

from omegaconf import OmegaConf
from pytorch_lightning import Trainer
from IPython.display import display
from tqdm import tqdm

from nemo.collections import nlp as nemo_nlp
from nemo.utils.exp_manager import exp_manager

In [13]:
import numpy as np
import torch
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel  

tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")  
model = AutoModel.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext").cuda()

In [14]:
import pandas as pd

In [18]:
df = pd.read_csv('DataLeo/umls_spa_cui_term_test.txt',sep='\|\|',header=None)

[NeMo W 2024-04-16 10:00:06 nemo_logging:349] /tmp/ipykernel_1473604/1043993813.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
      df = pd.read_csv('DataLeo/umls_spa_cui_term_test.txt',sep='\|\|',header=None)
    


In [20]:
df.columns=['Term','Code']

In [21]:
df.head()

,Term,Code
0,"""yemas "" tumorales:prthr:punto temporal:muestr...",C4482620
1,"""yemas"" de tumor:número aréico (por área):punt...",C4482621
2,% de actividad,C0439167
3,% de agregación,C0439168
4,% de captación,C0347980


In [26]:
all_names = list(df["Term"].values)

In [28]:
bs = 128 # batch size during inference
all_embs = []
for i in tqdm(np.arange(0, len(all_names), bs)):
    toks = tokenizer.batch_encode_plus(all_names[i:i+bs], 
                                       padding="max_length", 
                                       max_length=25, 
                                       truncation=True,
                                       return_tensors="pt")
    toks_cuda = {}
    for k,v in toks.items():
        toks_cuda[k] = v.cuda()
    cls_rep = model(**toks_cuda)[0][:,0,:] # use CLS representation as the embedding
    all_embs.append(cls_rep.cpu().detach().numpy())

all_embs = np.concatenate(all_embs, axis=0)

  0%|          | 0/9744 [00:00<?, ?it/s]

In [29]:
d = all_embs.shape[1]
d

768

In [35]:
quantizer = faiss.IndexFlatL2(768)
index = faiss.IndexIVFFlat(quantizer, 768, 50)
# index = faiss.index_cpu_to_all_gpus(index)
index.train(all_embs)

In [36]:
index.add(all_embs)

In [37]:
index.ntotal

1247174

In [46]:
faiss.write_index(index,'dataLeo_sap_bert_index')

In [38]:
k=5

In [41]:
test_data = pd.read_csv('DataLeo/test_norm_ref.tsv',sep='\t',header=None)

In [44]:
test_data.loc[:,4][0:10]

0                                nicotina
1                                    iPTH
2                          fosfato sérico
3                                     HDL
4                         trombocitopenia
5                            hemodiálisis
6                            nicotinamida
7                         hiperfosfatemia
8                            hemodiálisis
9    evaluaron la eficacia y la seguridad
Name: 4, dtype: object

In [55]:
all_names = list(test_data.loc[:,4])
bs = 128 # batch size during inference
all_embs = []
for i in tqdm(np.arange(0, len(all_names), bs)):
    toks = tokenizer.batch_encode_plus(all_names[i:i+bs], 
                                       padding="max_length", 
                                       max_length=25, 
                                       truncation=True,
                                       return_tensors="pt")
    toks_cuda = {}
    for k,v in toks.items():
        toks_cuda[k] = v.cuda()
    cls_rep = model(**toks_cuda)[0][:,0,:] # use CLS representation as the embedding
    all_embs.append(cls_rep.cpu().detach().numpy())

all_embs = np.concatenate(all_embs, axis=0)

  0%|          | 0/88 [00:00<?, ?it/s]

In [56]:
k = 5

In [57]:
D, I = index.search(all_embs, k) 

In [58]:
I

array([[ 791489,  722719,  791491,  791496,  791460],
       [ 653295,  989134,  989146,  656525,  855832],
       [ 513638, 1084669,  514595,  512616,  514229],
       ...,
       [1089285, 1089286, 1089282, 1089283, 1089284],
       [ 860516,  341964,  860893,  532591,  890882],
       [1030135, 1030136, 1030137, 1031804, 1033241]])

In [61]:
codes = df['Code'].iloc[I[0]].values

In [62]:
codes

array(['C0028040', 'C0202431', 'C0238311', 'C3533135', 'C0028027'],
      dtype=object)

In [63]:
test_data.loc[:,5][0] in codes

True

In [67]:
aciertosTop1 = 0
aciertosTop5 = 0

for i,x in tqdm(enumerate(I)):
    codes = df['Code'].iloc[x].values
    code = test_data.loc[:,5][i]
    if (code==codes[0]):
        aciertosTop1 += 1
    if (code in codes):
        aciertosTop5 += 1

0it [00:00, ?it/s]

In [68]:
print("Accuracy Top 1 %f" % (aciertosTop1/len(I)))
print("Accuracy Top 5 %f" % (aciertosTop5/len(I)))

Accuracy Top 1 0.896361
Accuracy Top 5 1.000000


In [69]:
5

5

In [70]:
test_data_uncert = pd.read_csv('DataLeo/test_norm_ref_with_uncert.tsv',sep='\t',header=None)

In [72]:
all_names = list(test_data_uncert.loc[:,4])
bs = 128 # batch size during inference
all_embs = []
for i in tqdm(np.arange(0, len(all_names), bs)):
    toks = tokenizer.batch_encode_plus(all_names[i:i+bs], 
                                       padding="max_length", 
                                       max_length=50, 
                                       truncation=True,
                                       return_tensors="pt")
    toks_cuda = {}
    for k,v in toks.items():
        toks_cuda[k] = v.cuda()
    cls_rep = model(**toks_cuda)[0][:,0,:] # use CLS representation as the embedding
    all_embs.append(cls_rep.cpu().detach().numpy())

all_embs = np.concatenate(all_embs, axis=0)

  0%|          | 0/91 [00:00<?, ?it/s]

In [73]:
k=5

In [74]:
D, I = index.search(all_embs, k) 

In [76]:
aciertosTop1 = 0
aciertosTop5 = 0

for i,x in tqdm(enumerate(I)):
    codes = df['Code'].iloc[x].values
    code = test_data_uncert.loc[:,5][i]
    if (code==codes[0]):
        aciertosTop1 += 1
    if (code in codes):
        aciertosTop5 += 1

0it [00:00, ?it/s]

In [77]:
print("Accuracy Top 1 %f" % (aciertosTop1/len(I)))
print("Accuracy Top 5 %f" % (aciertosTop5/len(I)))

Accuracy Top 1 0.868947
Accuracy Top 5 0.970016
